# IMPORTS

In [1]:
import tensorflow as tf
from sklearn.metrics import precision_recall_curve
import numpy as np
import os
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.metrics import f1_score, classification_report
import json
from google.colab import runtime
from google.colab import drive
drive.mount('/content/drive')
import gc

Mounted at /content/drive


# DATASET

In [2]:
batch_size = 1
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
categories = ["Normal","Osteopenia", "Osteoporosis"]
TfDatasetsPath = '/content/drive/MyDrive/CSEN-240-Project/TfDatasetsFull'

valDataset = tf.data.Dataset.load(TfDatasetsPath + "/validDataset")
testDataset = tf.data.Dataset.load(TfDatasetsPath + "/testDataset")

valDataset = valDataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
testDataset = testDataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# MODELS

In [3]:
thresholdsPath = '/content/drive/MyDrive/CSEN-240-Project/Thresholds/'
kerasModelsPath = '/content/drive/MyDrive/CSEN-240-Project/KerasModels/'

In [4]:
modelNames = os.listdir(kerasModelsPath)

In [5]:
models = []
for modelName in modelNames:
    modelPath = kerasModelsPath + modelName
    model = tf.keras.models.load_model(modelPath)
    results = model.evaluate(valDataset, return_dict=True)
    models.append([modelName, results['auc']])
models

1080/1080 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.8897 - auc: 0.9595 - loss: 0.4721 - precision: 0.8906 - recall: 0.8885
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.8891 - auc: 0.9684 - loss: 0.3453 - precision: 0.8901 - recall: 0.8839
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.8429 - auc: 0.9464 - loss: 0.5785 - precision: 0.8424 - recall: 0.8382
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 51s 18ms/step - accuracy: 0.8921 - auc: 0.9644 - loss: 0.4197 - precision: 0.8929 - recall: 0.8909
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8796 - auc: 0.9704 - loss: 0.3602 - precision: 0.8822 - recall: 0.8780
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 55s 23ms/step - accuracy: 0.8667 - auc: 0.9558 - loss: 0.4461 - precision: 0.8707 - recall: 0.8594
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 66s 34ms/step - accuracy: 0.8454 - auc: 0.9465 - loss: 0.5670 - precision: 0.8462 - recall: 0.8430
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8460 - auc: 0.9498 - loss: 0.4936 

[['Xception.keras', 0.9669954180717468],
 ['VGG16.keras', 0.9701579213142395],
 ['ResNet152V2.keras', 0.9537107348442078],
 ['DenseNet201.keras', 0.9715808033943176],
 ['InceptionResNetV2.keras', 0.975060224533081],
 ['EfficientNetV2L.keras', 0.962633490562439],
 ['NASNetLarge.keras', 0.9576725363731384],
 ['MobileNet.keras', 0.9579649567604065]]

In [6]:
models.sort(key=lambda x: x[1], reverse=True)
modelNames = [model[0] for model in models]
modelNames

['InceptionResNetV2.keras',
 'DenseNet201.keras',
 'VGG16.keras',
 'Xception.keras',
 'EfficientNetV2L.keras',
 'MobileNet.keras',
 'NASNetLarge.keras',
 'ResNet152V2.keras']

# ENSEMBLING FUNCTION

In [7]:
def ensembleModels(predictedClasses):
    votedPredictions = []
    numModels = len(predictedClasses)
    samples = len(predictedClasses[0])
    for i in range(samples):
        count0, count1, count2 = 0, 0, 0
        for j in range(numModels):
            if predictedClasses[j][i] == 0:
                count0 += 1
            elif predictedClasses[j][i] == 1:
                count1 += 1
            else:
                count2 += 1
        if count0 > count1 and count0 > count2:
            votedPredictions.append(0)
        elif count1 > count0 and count1 > count2:
            votedPredictions.append(1)
        else:
            votedPredictions.append(2)
    return votedPredictions

def customPredict(yPred, thresholds):
    predictions = np.zeros_like(yPred)
    for i in thresholds.keys():
        threshold = thresholds[i]
        predictions[:, i] = (yPred[:, i] >= threshold).astype(int)
    predictions = np.argmax(predictions * yPred, axis=1)
    return predictions

def predict(modelNames, testDataset):
    predictedClasses = []
    for modelName in modelNames:
        name = modelName.split('.')[0]
        modelPath = kerasModelsPath + modelName
        model = tf.keras.models.load_model(modelPath)
        predictions = model.predict(testDataset)
        thresholdPath = thresholdsPath + name + '.json'
        with open(thresholdPath, 'r') as f:
            threshold = json.load(f)
            threshold = {int(k): v for k, v in threshold.items()}
        predicted = customPredict(predictions, threshold)
        predictedClasses.append(predicted)
        tf.keras.backend.clear_session()
        del model
        gc.collect()
    return ensembleModels(predictedClasses)

In [8]:
bestAverageF1 = -1
topK = -1
bestModels = None

yTrue = []
for xBatch, yBatch in testDataset:
    yTrue.extend(np.argmax(yBatch.numpy(), axis=1))

for k in range(2, len(modelNames)+1):
    print(f"Top {k} : {modelNames[:k]}")
    yPred = predict(modelNames[:k], testDataset)
    f1 = f1_score(yTrue, yPred, average='weighted')
    if f1 > bestAverageF1:
        bestAverageF1 = f1
        topK = k
        bestModels = modelNames[:k]
    report = classification_report(yTrue, yPred, target_names=categories)
    print("F1 Score: ", f1)
    print(report)
bestAverageF1, topK, bestModels

Top 2 : ['InceptionResNetV2.keras', 'DenseNet201.keras']
540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step
F1 Score:  0.937309102063909
              precision    recall  f1-score   support

      Normal       0.94      0.94      0.94       180
  Osteopenia       0.98      0.94      0.96       180
Osteoporosis       0.90      0.93      0.92       180

    accuracy                           0.94       540
   macro avg       0.94      0.94      0.94       540
weighted avg       0.94      0.94      0.94       540

Top 3 : ['InceptionResNetV2.keras', 'DenseNet201.keras', 'VGG16.keras']
540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
F1 Score:  0.9384353505274602
              precision    recall  f1-score   support

      Normal       0.91      0.98      0.94       180
  Osteopenia       0.95      0.97      0.96       180
Osteoporosis       0.96      0.87      0.92       180



(0.9499121286101828,
 4,
 ['InceptionResNetV2.keras',
  'DenseNet201.keras',
  'VGG16.keras',
  'Xception.keras'])

# EVALUATION

In [9]:
yTrue = []
for xBatch, yBatch in testDataset:
    yTrue.extend(np.argmax(yBatch.numpy(), axis=1))

yPred = predict(bestModels, testDataset)

report = classification_report(yTrue, yPred, target_names=categories)
print(report)

540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 26s 18ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
540/540 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step
              precision    recall  f1-score   support

      Normal       0.94      0.97      0.95       180
  Osteopenia       0.97      0.96      0.97       180
Osteoporosis       0.94      0.92      0.93       180

    accuracy                           0.95       540
   macro avg       0.95      0.95      0.95       540
weighted avg       0.95      0.95      0.95       540



In [10]:
runtime.unassign()